In [1]:
%pip install ..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=94446 sha256=67d437e2bdef58fc58673fc56b96b1746ee843dccfdc005de522d38cfd8e63d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-cq6t2j_z/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.utils import set_seed

set_seed(0)

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


100%|██████████| 11996/11996 [00:01<00:00, 11444.68it/s]


In [5]:
from spuco.invariant_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


100%|██████████| 48004/48004 [00:04<00:00, 11849.41it/s]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [ ]:
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

In [ ]:
from torch.optim import SGD

erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [ ]:
erm.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()